In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Shoefly AB Testing.csv


In [3]:
import pandas as pd

In [7]:
ad_clicks = pd.read_csv('/kaggle/input/Shoefly AB Testing.csv')
print(ad_clicks.head(5))

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


The dataset is about a made-up online shoe store, ShoeFly.com. The store has two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google.

In [8]:
click_by_source= ad_clicks.groupby('utm_source').user_id.count().reset_index
print(click_by_source)

<bound method Series.reset_index of utm_source
email       255
facebook    504
google      680
twitter     215
Name: user_id, dtype: int64>


Banner ads on Google and Facebook got the most views, significantly more than ads in emails

In [10]:
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.notnull()

clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

clicks_pivot = clicks_by_source.pivot(
  columns='is_click',
  index= 'utm_source',
  values= 'user_id').reset_index()

mylambda= lambda row: 100*row[True]/(row[True]+row[False])
clicks_pivot['percent_clicked']= clicks_pivot.apply(mylambda, axis=1)

print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80        31.372549
1          facebook    324   180        35.714286
2            google    441   239        35.147059
3           twitter    149    66        30.697674


Created 'is_click' column to identify if someone actually clicked on the ad that was displayed. True if ad_click_timestamp is not null and False otherwise. Once again, Facebook and Google ads had the highest click-through rates. This means people are more likely to click on ads on Facebook or Google than other sources.

In [11]:
count_group= ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print(count_group)

  experimental_group  user_id
0                  A      827
1                  B      827


The column experimental_group tells whether the user was shown Ad A or Ad B. The numbers of people shown both ads were the same: 827.

In [12]:
count_group2= ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()

count_group2_pivot=count_group2.pivot(
columns='is_click',
index='experimental_group',
values='user_id'
).reset_index()

count_group2_pivot['percent_clicked']= 100*count_group2_pivot[True]/827
print (count_group2_pivot)

is_click experimental_group  False  True  percent_clicked
0                         A    517   310        37.484885
1                         B    572   255        30.834341


Using the column is_click that we defined earlier, we can tell that a larger percentage of users clicked on Ad A.

In [13]:
a_clicks= ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks= ad_clicks[ad_clicks.experimental_group == 'B']

a_clicks2= a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
a_clicks2_pivot = a_clicks2.pivot(
  columns='is_click', 
  index='day',
  values='user_id'
  ).reset_index() 
a_clicks2_pivot['percent']= a_clicks2_pivot.apply(mylambda, axis=1)

b_clicks2= b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
b_clicks2_pivot= b_clicks2.pivot(
  columns= 'is_click',
  index= 'day',
  values= 'user_id').reset_index()
b_clicks2_pivot['percent']= b_clicks2_pivot.apply(mylambda, axis=1)

print(a_clicks2_pivot)
print(b_clicks2_pivot)

is_click            day  False  True    percent
0            1 - Monday     70    43  38.053097
1           2 - Tuesday     76    43  36.134454
2         3 - Wednesday     86    38  30.645161
3          4 - Thursday     69    47  40.517241
4            5 - Friday     77    51  39.843750
5          6 - Saturday     73    45  38.135593
6            7 - Sunday     66    43  39.449541
is_click            day  False  True    percent
0            1 - Monday     81    32  28.318584
1           2 - Tuesday     74    45  37.815126
2         3 - Wednesday     89    35  28.225806
3          4 - Thursday     87    29  25.000000
4            5 - Friday     90    38  29.687500
5          6 - Saturday     76    42  35.593220
6            7 - Sunday     75    34  31.192661


We are curious to see if the clicks for Ad A or B might have changed by day of the week. We created two dataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively. Subsequently, for each group, we calculated the percent of users who clicked on the ad by day.  
For ad A, users are more likely to click on it from Thursday to Sunday. For ad B,Tuesday and Saturday see the highest percentages of click rates. In summary, I recommend the company using ad A and focusing on displaying it more from Thursday to Sunday, while reducing the frequency on Wednesday.Also it could be more effective to display only ad B on Tuesday, since it has a sizably higher click rate than ad A. 